**Human Language Technologies Project**

**Authors:** Dalla Noce Niko, Ristori Alessandro

#HLT Project

This work is higly based on the tensorflow tutorial https://www.tensorflow.org/text/tutorials/transformer, our aim was to introduce BERT as an encoder in the model and try combinations with different architectures (both RNNs and transformers).

##Setup

We need to install the transformers package to use the models and tokenizers from HuggingFace.

In [1]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 48.2 MB/s 
     |████████████████████████████████| 59 kB 6.2 MB/s 
     |████████████████████████████████| 3.3 MB 38.1 MB/s 
     |████████████████████████████████| 895 kB 52.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.6 MB/s 


Import the libraries needed for the project to work.

In [2]:
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras import layers

The model training is going to run on TPUs since they are the optimized for working with tensors, to do so we need colab to assign us as much TPUs as possible.

In [3]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.53.152.178:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.53.152.178:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [4]:
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

##Preprocess the dataset

Let's define the method to preprocess the anki dataset.

In [ ]:
def create_dataset_anki(name: str, preprocessed:bool) -> (list, list):
    with open(name, encoding="UTF-8") as datafile:
        src_set = list()
        dst_set = list()
        for sentence in datafile:
            sentence = sentence.split("\t")
            src_set.append(sentence[0])
            if preprocessed:
                dst_set.append(sentence[1].split("\n")[0])
            else:
                dst_set.append(sentence[1])

    return src_set, dst_set

We assume that the dataset was uploaded on colab with a zip file, we need to extract it and then we can build our lists using the previous method.

In [ ]:
import zipfile
with zipfile.ZipFile("dataset_anki_it.zip", 'r') as zip_ref:
    zip_ref.extractall("")

en_set, it_set = create_dataset_anki("ita_preprocessed.txt", True)
print("The corpus' size is: {0}".format(len(en_set)))

The corpus' size is: 352040


##Build the dataset

Before we create the dataset from our lists, we have to tokenize each sentence from the corpus by using the BERT tokenizer for english and the one for italian. Moreover we can get the number of tokens for both source and target.

In [ ]:
from transformers import BertTokenizer

logging.getLogger("transformers").setLevel(logging.ERROR)  # suppress warning for transformers

# Create the tokenizers and get the number of tokens
tokenizer_en = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer_it = BertTokenizer.from_pretrained("dbmdz/bert-base-italian-cased")
v_size_en = tokenizer_en.vocab_size
v_size_it = tokenizer_it.vocab_size

print("Number of tokens for the english dataset: {0}".format(v_size_en))
print("Number of tokens for the italian dataset: {0}".format(v_size_it))

Number of tokens for the english dataset: 30522
Number of tokens for the italian dataset: 31102


Let's calculate the max number of tokens allowed, this number is taken such that 99% of the sentences in the dataset are fully tokenized.

In [ ]:
def set_max_tokens(dataset: list(), language: str = "en") -> int:
    len_sentences = [len(sentence.split()) for sentence in dataset[:351000]]
    # plt.boxplot(len_sentences)
    mean_len_sentences = np.mean(len_sentences)
    print("{0} dataset average sentence length: {1}".format(language, mean_len_sentences))
    max_length = int(mean_len_sentences + 3 * np.std(len_sentences))
    print("{0} dataset max length allowed: {1}".format(language, max_length))
    return max_length

max_length_en = set_max_tokens(en_set, "en")
max_length_it = set_max_tokens(it_set, "it")

en dataset average sentence length: 5.492988603988604
en dataset max length allowed: 11
it dataset average sentence length: 5.35585754985755
it dataset max length allowed: 11


Tokenize the source and target dataset.

In [ ]:
# Tokenize the dataset
tokens_en = tokenizer_en(en_set[:351000], add_special_tokens=True,
                          truncation=True, padding="max_length", return_attention_mask=True,
                          return_tensors="tf", max_length=max_length_en).data["input_ids"]
tokens_it = tokenizer_it(it_set[:351000], add_special_tokens=True,
                          truncation=True, padding="max_length", return_attention_mask=True,
                          return_tensors="tf", max_length=max_length_it+1).data["input_ids"]

In [ ]:
for _ in range(5):
  i = np.random.randint(len(tokens_en))
  print("En sentence: {0}\nTokenized sentence: {1}".format(en_set[i], tokens_en[i]))
  print("It sentence: {0}\nTokenized sentence: {1}\n".format(it_set[i], tokens_it[i]))

En sentence: Have you lost the receipt?
Tokenized sentence: [  101  2031  2017  2439  1996 24306  1029   102     0     0     0]
It sentence: Hai perso la ricevuta?
Tokenized sentence: [  102   537  3162   143 15721  2937   103     0     0     0     0     0]

En sentence: Tom could speak French.
Tokenized sentence: [ 101 3419 2071 3713 2413 1012  102    0    0    0    0]
It sentence: Tom riusciva a parlare il francese.
Tokenized sentence: [  102  5247 19354   111  1691   152  2194   687   103     0     0     0]

En sentence: You've got to come tomorrow.
Tokenized sentence: [ 101 2017 1005 2310 2288 2000 2272 4826 1012  102    0]
It sentence: Dovete venire domani.
Tokenized sentence: [ 102 5570 2698 3156  687  103    0    0    0    0    0    0]

En sentence: How far is it from here to your house?
Tokenized sentence: [ 101 2129 2521 2003 2009 2013 2182 2000 2115 2160  102]
It sentence: Quanto dista da qui a casa tua?
Tokenized sentence: [ 102  677 5777  165  495  111  695  878 2937  103  

Then we build the tf dataset and split it into training, validation and test sets.

In [ ]:
def split_set(dataset: tf.data.Dataset,
              tr: float = 0.8,
              val: float = 0.1,
              ts: float = 0.1,
              shuffle: bool = True) -> (tf.data.Dataset, tf.data.Dataset, tf.data.Dataset):
    if tr+val+ts != 1:
        raise ValueError("Train, validation and test partition not allowed with such splits")

    dataset_size = dataset.cardinality().numpy()
    if shuffle:
        dataset = dataset.shuffle(dataset_size)

    tr_size = int(tr * dataset_size)
    val_size = int(val * dataset_size)

    tr_set = dataset.take(tr_size)
    val_set = dataset.skip(tr_size).take(val_size)
    ts_set = dataset.skip(tr_size).skip(val_size)
    return tr_set, val_set, ts_set

In [ ]:
# Build the dataset and split it in train, validation and test
dataset = tf.data.Dataset.from_tensor_slices((tokens_en, tokens_it))  # build the tf dataset
tr_set, val_set, ts_set = split_set(dataset, 0.8, 0.1, 0.1)  # split the tf dataset
print("Training set size: {0}".format(len(tr_set)))
print("Validation set size: {0}".format(len(val_set)))
print("Test set size: {0}".format(len(ts_set)))

Training set size: 280800
Validation set size: 35100
Test set size: 35100


After we have built our development and test set, we need to split the first one (both training and validation) in batches.

In [ ]:
def __format_dataset(eng, ita):
    return ({"encoder_inputs": eng, "decoder_inputs": ita[:, :-1],}, ita[:, 1:])

def make_batches(dataset_src_dst: tf.data.Dataset, batch_size: int) -> tf.data.Dataset:
    dataset = dataset_src_dst.batch(batch_size)
    dataset = dataset.map(__format_dataset)
    return dataset.prefetch(tf.data.experimental.AUTOTUNE).cache()

# def make_batches(dataset_src_dst: tf.data.Dataset, batch_size: int):
#    return dataset_src_dst.cache().batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
with strategy.scope():
  tr_batches = make_batches(tr_set, 128)
  val_batches = make_batches(val_set, 128)

In [ ]:
for src, dst in tr_batches.take(1):
  print("encoder inputs shape: {0}".format(src["encoder_inputs"].shape))
  print("decoder inputs shape: {0}".format(src["decoder_inputs"].shape))
  print("targets shape: {0}".format(dst.shape))

encoder inputs shape: (128, 11)
decoder inputs shape: (128, 11)
targets shape: (128, 11)


##Positional embeddings layer

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

##Encoder and decoder

As we know from the HLT course, NMT models are based on the encoder-decoder paradigm, therefore we have to build both. We based the architecture of those layers from the paper "Attention is all you need" from Vaswani et al.

###Encoder

The single layer of the encoder transformer.

In [ ]:
class EncoderLayer(layers.Layer):

    def __init__(self, layers_size: int, dense_size: int, num_heads: int, **kwargs) -> None:
        super(EncoderLayer, self).__init__(**kwargs)
        
        self.layers_size = layers_size
        self.dense_size = dense_size
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(num_heads, layers_size)
        self.dense_proj = tf.keras.Sequential(
            [layers.Dense(dense_size, activation="relu"), layers.Dense(layers_size)]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, mask=None) -> tf.Tensor:
        if mask is not None:  
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        else:
            print("Mask not built")
            assert False
        
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [ ]:
class EncoderTransformer(layers.Layer):

    def __init__(self,
                 num_layers: int,
                 layers_size: int,
                 dense_size: int,
                 num_heads: int,
                 dropout: float = 0.1) -> None:
        super(EncoderTransformer, self).__init__()

        self.layers_size = layers_size
        self.num_layers = num_layers
        self.pos_embedding = PositionalEmbedding(max_length_en, v_size_en, layers_size)
        self.enc_layers = [EncoderLayer(layers_size, dense_size, num_heads) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, mask=None) -> tf.Tensor:
        # src_embeddings = PositionalEmbedding(max_length_en, v_size_en, layers_size)(src_embeddings)
        src_embeddings = self.pos_embedding(inputs)
        enc_out = self.dropout(src_embeddings)
        for i in range(self.num_layers):
            enc_out = self.enc_layers[i](enc_out)

        return enc_out  # (batch_size, input_seq_len, layers_size)

###Decoder

The single layer of the decoder transformer.

In [ ]:
class DecoderLayer(layers.Layer):

    def __init__(self, layers_size: int, dense_size: int, num_heads: int, **kwargs) -> None:
        super(DecoderLayer, self).__init__(**kwargs)
        
        self.layers_size = layers_size
        self.dense_size = dense_size
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads, layers_size)
        self.attention_2 = layers.MultiHeadAttention(num_heads, layers_size)
        self.dense_proj = tf.keras.Sequential(
            [layers.Dense(dense_size, activation="relu"), layers.Dense(layers_size)]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, encoder_outputs: tf.Tensor, mask=None) -> tf.Tensor:
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [ ]:
class DecoderTransformer(layers.Layer):

    def __init__(self,
                 num_layers: int,
                 layers_size: int,
                 dense_size: int,
                 num_heads: int,
                 dropout=0.1) -> None:
        super(DecoderTransformer, self).__init__()

        self.layers_size = layers_size
        self.num_layers = num_layers
        self.pos_embedding = PositionalEmbedding(max_length_it, v_size_it, layers_size)
        self.dec_layers = [DecoderLayer(layers_size, dense_size, num_heads) for _ in range(num_layers)]
        self.dropout = layers.Dropout(dropout)
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, enc_output: tf.Tensor, mask=None) -> tf.Tensor:
        dst_embeddings = self.pos_embedding(inputs)
        dec_output = self.dropout(dst_embeddings)
        for i in range(self.num_layers):
            dec_output = self.dec_layers[i](dec_output, enc_output)

        return dec_output

##Build the model

In [ ]:
layers_size = 512
num_layers = 6
dense_size = 2048
num_heads = 8

def create_model():
    # Encoder
    encoder_inputs = tf.keras.Input(shape=(None,), dtype="int32", name="encoder_inputs")
    encoder_outputs = EncoderTransformer(num_layers, layers_size, dense_size, num_heads)(encoder_inputs)

    # Decoder
    decoder_inputs = tf.keras.Input(shape=(None,), dtype="int32", name="decoder_inputs")
    encoded_seq_inputs = tf.keras.Input(shape=(None, layers_size), name="decoder_state_inputs")
    decoder_outputs = DecoderTransformer(num_layers, layers_size, dense_size, num_heads)(decoder_inputs, encoded_seq_inputs)
    decoder_outputs = layers.Dropout(0.4)(decoder_outputs)
    decoder_outputs = layers.Dense(v_size_it, activation="softmax")(decoder_outputs)
    decoder = tf.keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs, name="decoder_transformer")
    # decoder.summary()

    decoder_outputs = decoder([decoder_inputs, encoder_outputs])
    transformer = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs, name="transformer")
    # transformer.summary()
    return transformer

In [ ]:
with strategy.scope():
    opt = tf.keras.optimizers.Nadam(learning_rate = 0.0001)
    transformer = create_model()
    transformer.summary()
    transformer.compile(opt, loss = "sparse_categorical_crossentropy", metrics=["accuracy"])

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
encoder_transformer_2 (EncoderT (None, None, 256)    17282816    encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_transformer (Functional (None, None, 31102)  31736702    decoder_inputs[0][0]             
                                                                 encoder_transformer_2[0

##Train and evaluate the model

In [ ]:
transformer.fit(tr_batches, epochs=20, validation_data = val_batches)

Epoch 1/10


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 11) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 11) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 11) dtype=int32>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 11) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 11) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 11) dtype=int32>]


2194/2194 [==============================] - ETA: 0s - loss: 2.7322 - accuracy: 0.5120

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 11) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 11) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 11) dtype=int32>]


2194/2194 [==============================] - 208s 64ms/step - loss: 2.7322 - accuracy: 0.5120 - val_loss: 1.6538 - val_accuracy: 0.6338
Epoch 2/10
2194/2194 [==============================] - 99s 45ms/step - loss: 1.3916 - accuracy: 0.6839 - val_loss: 1.0321 - val_accuracy: 0.7474
Epoch 3/10
2194/2194 [==============================] - 99s 45ms/step - loss: 0.9543 - accuracy: 0.7594 - val_loss: 0.7287 - val_accuracy: 0.8004
Epoch 4/10
2194/2194 [==============================] - 98s 45ms/step - loss: 0.7230 - accuracy: 0.7984 - val_loss: 0.5629 - val_accuracy: 0.8290
Epoch 5/10
2194/2194 [==============================] - 100s 45ms/step - loss: 0.5855 - accuracy: 0.8218 - val_loss: 0.4629 - val_accuracy: 0.8469
Epoch 6/10
2194/2194 [==============================] - 100s 45ms/step - loss: 0.4962 - accuracy: 0.8372 - val_loss: 0.3961 - val_accuracy: 0.8598
Epoch 7/10
2194/2194 [==============================] - 100s 46ms/step - loss: 0.4350 - accuracy: 0.8486 - val_loss: 0.3518 - val_ac

In [ ]:
ts_loss, ts_accuracy = transformer.evaluate(make_batches(ts_set, 128))
print("Test loss: {0}\nTest accuracy: {1}".format(ts_loss, ts_accuracy))

275/275 [==============================] - 7s 18ms/step - loss: 0.2828 - accuracy: 0.8813
Test loss: 0.2828167974948883
Test accuracy: 0.8812545537948608


##Translator

In [ ]:
max_decoded_sentence_length = 10

def translate(input_sentence):
    tokenized_input_sentence = tokenizer_en(input_sentence, return_tensors='tf', add_special_tokens=True, max_length = max_length_en, padding='max_length').data["input_ids"]
    decoded_sentence = "[CLS]"
    list_tokens=[]
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = tokenizer_it(decoded_sentence, return_tensors='tf', add_special_tokens=False, max_length = max_length_it, padding='max_length').data['input_ids']
        predictions = transformer.predict([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = tokenizer_it.ids_to_tokens[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[SEP]":
            break
        
        list_tokens.append(sampled_token) 
      
    return decoded_sentence, tokenizer_it.convert_tokens_to_string(list_tokens)

In [ ]:
tokens , translated = translate("This task is very hard.")
print(translated)

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 11) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 11) dtype=int32>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 11) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 13) dtype=int32>]


ValueError: ignored